In [1]:
import xarray
import metview as mv

# Introduction to Model-level datasets

This particular ERA5 dataset contains atmospheric variables on their native horizontal and vertical grid.  There has been no interpolation or other processing.  This makes the data very useful for advanced users, but it's intimidating for novice users.  In this section, we will briefly describe the CO ERA5 model-level datasets and the horizontal and vertical grids.  


## Horizontal Grids
The model-level datasets don't use latitude-longitude grids with a fixed spacing.  Instead, they use two systems for horizontal leves, spherical harmonics and Gaussian grids.  Our discussion of these grids will be brief.  If you would like a more detailed reference, we strongly recommend this graduate-level textbook, _Fundamentals of Numerical Weather Prediction_ by Jean Coffier (Cambridge University Press, 2011)  

### Spherical Harmonics

A field on a spherical surface can be represented as the sum of a series of functions, much in the same way a one-dimensional function can be represented as the sum of a Fourier series.  It can be shown that such a series, Y,  must satisfy this relationship, $\nabla^2Y_n^m(\lambda,\mu)=-\frac{n(n+1)}{a^2}Y_n^m(\lambda, \mu)$, where $\lambda$ is the longitude, $\mu$ is the sine of the latitude, and a is the radius of the Earth.  We can then show tha $Y_n^m(\lambda, mu)=P_n^m(\mu)\exp(im\lambda)$, where $P_n^m(\mu)$ is the [Legendre polynomial](https://en.wikipedia.org/wiki/Legendre_polynomials).  From these, we see that m is a zonal wavenumber of a periodic function along a line of latitude, and that n is the total wavenumber.  

We can then express a field, A, as $A(\lambda,\mu) = \sum_{n=\left|m\right|)}^{\infty}\sum_{-\infty}^{\infty}A_{n,m}Y_n^m(\lambda,\mu)$, where $A_n^{m}$ are the spectral coefficients of A.  In practice, n and m are finite, this is called "truncating" the series.    

#### Gaussian Grid

Given that we are repesenting $A(\lambda,\mu)$ as the sum of a spherical harmonics, the discretization of A takes the form of a Gaussian grid.  A Gaussian grid is a set of latitude circles with number of points J, and the number of circles is K.  The spacing in longitude is constant across all circles, but the spacing in longitude decreases as one moves poleward from the equator.  The latitude for each circle depends on the Legendre polynomial.  

##### The Reduced Gaussian Grid

As we move towards the poles, the physical distance between grid points on a line of latitude decreases.  For computational efficiency, we can decrease the number of grid points in the lines of latitude as we approach this poles.  This is known as a reduced-Gaussian grid [Hortel and Simmons (1991)](https://journals.ametsoc.org/view/journals/mwre/119/4/1520-0493_1991_119_1057_uorggi_2_0_co_2.xml).  

Variables that are smoothly continous and do not have physical bounds are best stored as spherical harmonics. This would include fields like temperature and divergence.   Variables that have sharp discontinuities and/or lower physical bounds should be stored in reduced Gaussian grids.  This would include fields like specific humidity and ozone mixing ratio.  

## Vertical Grid
If the world was a perfect sphere, then the vertical grid for a weather model could be a simple set of levels above ground.  However, the presence of topography complicates things. (See Coiffier for more details).  ERA5 uses a system known as hybrid model levels for it's vertical grid ([Simmons and Burridge (1981)](https://journals.ametsoc.org/view/journals/mwre/109/4/1520-0493_1981_109_0758_aeaamc_2_0_co_2.xml)).  The model levels follow the earth's surface near the ground, but as one goes higher, the model levels transition to fixed pressure levels.  The pressure at a specific model level depends on the surface pressure beneath it.  This means that the pressure at model level 50 over Tibet is not going to be the same as the pressure at model level 50 over the ocean.  

## The datasets
### Single-level-surface, (spherical harmonics)
- Surface geopotential (m^2/s^2)(zs)
- Logarithm of Surface Pressure (lnsp)
 

In [2]:
ml_surface = xarray.open_zarr(
    'gs://gcp-public-data-arco-era5/co/single-level-surface.zarr/',
    chunks={'time': 48},
    consolidated=True,
)

print("Model surface dataset size {} TiB".format(ml_surface.nbytes/(1024**4)))
ml_surface

Model surface dataset size 1.1164016313996399 TiB


<xarray.Dataset>
Dimensions:     (time: 374016, values: 410240)
Coordinates:
    hybrid      float64 ...
    step        timedelta64[ns] ...
  * time        (time) datetime64[ns] 1979-01-01 ... 2021-08-31T23:00:00
    valid_time  (time) datetime64[ns] dask.array<chunksize=(48,), meta=np.ndarray>
Dimensions without coordinates: values
Data variables:
    lnsp        (time, values) float32 dask.array<chunksize=(48, 410240), meta=np.ndarray>
    z           (time, values) float32 dask.array<chunksize=(48, 410240), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.7
    GRIB_centre:               ecmf
    GRIB_centreDescription:    European Centre for Medium-Range Weather Forec...
    GRIB_edition:              2
    GRIB_subCentre:            0
    history:                   2022-09-28T00:04 GRIB to CDM+CF via cfgrib-0.9...
    institution:               European Centre for Medium-Range Weather Forec...
    pangeo-forge:inputs_hash:  62dcc83733d69bb9c25d180532d8911856761df3c788e5...
    pangeo-forge:recipe_hash:  ef1235c98ebe08c72744fe0ec5611b2e307a1087ec8b26...
    pangeo-forge:version:      0.9.1

## Model level wind-related variables (spherical harmonics)
- Divergence (1/s) (d)
- Vorticity (1/s) (vo)
- Temperature (K) (t)
- Vertical velocity (Pa/s) (w)
  

In [3]:
ml_wind = xarray.open_zarr(
    'gs://gcp-public-data-arco-era5/co/model-level-wind.zarr/',
    chunks={'time': 48},
    consolidated=True,
)
print("Model wind dataset size {} TiB".format(ml_wind.nbytes/(1024**4)))
ml_wind

Model wind dataset size 305.8925611573068 TiB


<xarray.Dataset>
Dimensions:     (time: 374016, hybrid: 137, values: 410240)
Coordinates:
  * hybrid      (hybrid) float64 1.0 2.0 3.0 4.0 5.0 ... 134.0 135.0 136.0 137.0
    step        timedelta64[ns] ...
  * time        (time) datetime64[ns] 1979-01-01 ... 2021-08-31T23:00:00
    valid_time  (time) datetime64[ns] dask.array<chunksize=(48,), meta=np.ndarray>
Dimensions without coordinates: values
Data variables:
    d           (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 410240), meta=np.ndarray>
    t           (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 410240), meta=np.ndarray>
    vo          (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 410240), meta=np.ndarray>
    w           (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 410240), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.7
    GRIB_centre:               ecmf
    GRIB_centreDescription:    European Centre for Medium-Range Weather Forec...
    GRIB_edition:              2
    GRIB_subCentre:            0
    history:                   2022-10-09T17:47 GRIB to CDM+CF via cfgrib-0.9...
    institution:               European Centre for Medium-Range Weather Forec...
    pangeo-forge:inputs_hash:  7db2d90013176f19f18ca7aa85222a2c796c298afe6e44...
    pangeo-forge:recipe_hash:  ed5fbf407898def1e27d4e8965c2cdaa01b7ea8f2ac2e1...
    pangeo-forge:version:      0.9.1

## Model level moisture/ozone variables (reduced Gaussian Grid)
- Specific humidity (kg/kg) (q)
- Ozone mass mixing ratio (kg/kg) (o3)
- Specific cloud liquid water content (kg/kg) (clwc)
- Specific cloud ice water content (kg/kg) (ciwc)
- Specific cloud rain water content (kg/kg) (crwc)
- Specific cloud snow water content (kg/kg) (cswc)
- Fraction of cloud cover (0-1) (cc)

In [5]:
ml_moisture = xarray.open_zarr(
    'gs://gcp-public-data-arco-era5/co/model-level-moisture.zarr/',
    chunks={'time': 48},
    consolidated=True,
)
print("Model moisture dataset size {} TiB".format(ml_moisture.nbytes/(1024**4)))
ml_moisture

Model moisture dataset size 707.3467227025685 TiB


<xarray.Dataset>
Dimensions:     (time: 374016, hybrid: 137, values: 542080)
Coordinates:
  * hybrid      (hybrid) float64 1.0 2.0 3.0 4.0 5.0 ... 134.0 135.0 136.0 137.0
    latitude    (values) float64 dask.array<chunksize=(542080,), meta=np.ndarray>
    longitude   (values) float64 dask.array<chunksize=(542080,), meta=np.ndarray>
    step        timedelta64[ns] ...
  * time        (time) datetime64[ns] 1979-01-01 ... 2021-08-31T23:00:00
    valid_time  (time) datetime64[ns] dask.array<chunksize=(48,), meta=np.ndarray>
Dimensions without coordinates: values
Data variables:
    cc          (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 542080), meta=np.ndarray>
    ciwc        (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 542080), meta=np.ndarray>
    clwc        (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 542080), meta=np.ndarray>
    crwc        (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 542080), meta=np.ndarray>
    cswc        (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 542080), meta=np.ndarray>
    o3          (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 542080), meta=np.ndarray>
    q           (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 542080), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.7
    GRIB_centre:               ecmf
    GRIB_centreDescription:    European Centre for Medium-Range Weather Forec...
    GRIB_edition:              2
    GRIB_subCentre:            0
    history:                   2022-09-27T00:00 GRIB to CDM+CF via cfgrib-0.9...
    institution:               European Centre for Medium-Range Weather Forec...
    pangeo-forge:inputs_hash:  06482fd143803f9b1ae4d85151c399e57173179ee57bcf...
    pangeo-forge:recipe_hash:  7af5b330ea42d563af5c8b6d2606cb839306f90d83bfd2...
    pangeo-forge:version:      0.9.1

# Local Processing
For demonstration purposes, we're going to work with hourly slices of the ERA5 ML dataset.  It's possible to download an hourly slice in a reasonable amount of time over a home internet link.

In [6]:
# try each date...
datestring="1987-10-16T03"
datestring="1999-05-03T21"
datestring="2017-10-09T06"

surface_slice = ml_surface.sel(time=slice(datestring,datestring)).compute()
print("Surface slice size {} Gb".format(surface_slice.nbytes/(1024**3)))

wind_slice = ml_wind.sel(time=slice(datestring,datestring)).compute()
print("Wind slice size {} Gb".format(wind_slice.nbytes/(1024**3)))

moisture_slice = ml_moisture.sel(time=slice(datestring,datestring)).compute()
print("Moisture slice size {} Gb".format(moisture_slice.nbytes/(1024**3)))

Surface slice size 0.003056555986404419 Gb
Wind slice size 0.8374892175197601 Gb
Moisture slice size 1.9446884095668793 Gb


# Introduction to Metview

[Metview](https://metview.readthedocs.io/en/latest/index.html) is a meteorological software package developed by ECMWF and INPE.  It's used in research and operations to process and analyze forecast data produced by ECMWF.   Metview started as a graphical workstation, but now has a set of Python bindings for batch processing or use in Jupyter notebooks.  Metview's development history makes it well-suited for analyzing/processing ERA5 model-level data.

Metview uses a datatype known as a [fieldset](https://metview.readthedocs.io/en/latest/data_types/fieldset.html) to represent gridded datasets.  Typically, fieldsets are created by using the `read` procedure to load GRIB files.  It is possible to convert xarray's Datasets to metview's fieldsets using `dataset_to_fieldset`.  This procedure uses [cfgrib](https://github.com/ecmwf/cfgrib) to write GRIB messages that are then loaded as fieldsets.

In [9]:
def attribute_fix(ds):
    """Needed to fix a low-level bug in ecCodes.
    
    Sometimes, shortNames get overloaded in ecCodes's table. 
    To eliminate ambiguitity in their string matching, we 
    force ecCodes to make use of the paramId, which is a
    consistent source-of-truth.
    """
    for var in ds:
        attrs = ds[var].attrs
        result = attrs.pop('GRIB_cfName', None)
        result = attrs.pop('GRIB_cfVarName', None)
        result = attrs.pop('GRIB_shortName', None)
        ds[var].attrs.update(attrs)
    return ds

In [10]:
import metview as mv
wind_fieldset = mv.dataset_to_fieldset(attribute_fix(wind_slice).squeeze())
surface_fieldset = mv.dataset_to_fieldset(attribute_fix(surface_slice).squeeze())
moist_fieldset = mv.dataset_to_fieldset(attribute_fix(moisture_slice).squeeze())

# Interpolation
Interpolations in metview are done using the [`read`](https://metview.readthedocs.io/en/latest/api/functions/read.html) procedure.  The grid argument specifies the output grid of the new fieldset.  The format specification for reduced Gaussian grids is `N<number of latitudes in hemisphere>`. [Documentation on ECWMF's Gaussian grids](https://www.ecmwf.int/en/forecasts/documentation-and-support/gaussian_n320). ERA5 uses the reduced Gaussian grid `N320`.  For global latitude-longitude grids, the grid specification is `[dlat, dlon]`, where dlat is the spacing between latitudes, and dlon is the spacing between longitudes.  

In [11]:
surface_gg = mv.read(data=surface_fieldset,grid='N320')
wind_gg = mv.read(data=wind_fieldset,grid='N320')

# Converting divergence/vorticity to u/v winds 

Since the winds have a speed and direction, that means the wind is a vector.  In mathematical notation, we represent wind as $\vec{U}$  The meteorlogical convention is to the describe the wind blowing in the east-west direction as the zonal wind, or u-wind. The wind blowing in the north-south direction is known as the meridional wind, or v-wind.  As noted earlier, ERA5 uses spherical harmonics, and vectors cannot be directly represented in these harmonics.   Instead, the equations of motion predict the divergence, $D$, and horizontal vorticity, $\zeta$.   From vector calculus, we find that $D = \nabla \cdot \vec{U}$ and $\zeta = \nabla \times \vec{U}$.  The problem with this is that in the real world, we experiece and measure the wind, not divergerce and vorticity.

The Helmholtz relation states that we can compute the horizontal wind vector $\vec{U}$ from the scalar variables known as streamfunction, $\psi$, and velocity potential, $\chi$, using this equation, $\vec{U} = \hat{k} \times \nabla \psi + \nabla \chi$.  Streamfunction is defined as $\nabla^2 \psi = \zeta$, and velocity potential is $\nabla^2 \chi = D$.  In practice, we would use Poisson's equation to compute $\psi$ and $\chi$ from $D$ and $\zeta$ and then compute the winds using the Helmholtz relation.  Since we are using spherical harmonics, the differentiation and integration is transformed into summations of harmonic coefficients.

We don't have to implement our own conversion process, metview's [`uvwind`](https://metview.readthedocs.io/en/latest/gen_files/icon_functions/uvwind.html) procedure will do the conversion in spherical harmonics.  It requires a fieldset with d, and vo.  We set the truncation to 639 to match ERA5's truncation.  Once the conversion is finished, we then convert the u,v components from spectral space to a reduced Gaussian grid and then interpolate to a regular lat-lon grid.



In [12]:
uv_wind_spectral = mv.uvwind(data=wind_fieldset,truncation=639)
uv_wind_gg = mv.read(data=uv_wind_spectral,grid='N320')
uv_wind = mv.read(data=uv_wind_gg,grid=[0.25, 0.25])

# Metview computational functions
Metview has an extensive set of [functions](https://metview.readthedocs.io/en/latest/gen_files/toc/comp.html) to compute meteorological quantities.  These functions create new fieldsets that have all of the appropriate metadata.  As an example of this, we wish to compute the wind speed on model levels.  Mathematically, that can be represented at $\left|\vec{U}\right| = \sqrt{u^2+v^2}$.  We can use the function [`speed`](https://metview.readthedocs.io/en/latest/api/functions/speed.html) to do this, but we must pass in the u and v wind fieldsets separately. We do this by using [`select`](https://metview.readthedocs.io/en/latest/api/functions/select.html) to get the u and v winds separately from the uv_wind fieldset.  To select variables, you'll need the shortName which is can be found in the [ECMWF parameter database](https://apps.ecmwf.int/codes/grib/param-db/)

In [13]:
u_wind = uv_wind.select(shortName='u')
v_wind = uv_wind.select(shortName='v')

speed = mv.speed(u_wind,v_wind)

# Merging fieldsets
For simplicity's sake, we prefer to keep our different variables together in as few fieldsets as practical.  We can merge our fieldsets together using [`merge`](https://metview.readthedocs.io/en/latest/api/functions/merge.html).  On a practical note, if you have to merge several different fieldsets togther, it's best to merge the fieldsets containing the least number of fields together before merging with the largest fieldset.  This is because metview creates a temporary file for each fieldset and merging a large fieldset with many different fieldsets will create many large temporary files.  

In [14]:
uv_wind = mv.merge(uv_wind,speed)
uv_wind.describe()

parameter,typeOfLevel,level,date,time,step,number,paramId,class,stream,type,experimentVersionNumber
u,hybrid,"1,2,...",20171009,600,0,None,131,None,None,None,None
v,hybrid,"1,2,...",20171009,600,0,None,132,None,None,None,None
ws,hybrid,"1,2,...",20171009,600,0,None,10,None,None,None,None


# Computing the pressure on model levels

For ERA5's vertical coordinate system, the pressure is defined at the boundary between levels.  For the k'th level, the pressure at the boundary between the level k, and the next level k+1 is given by $p_{k+\frac{1}{2}} = A_{k+\frac{1}{2}} + B_{k+\frac{1}{2}}p_{s}$ where A and B are fixed coefficients depending on the model level configuration and $p_s$ is the pressure at the surface.  A and B can be found in the `GRIB_pv` attribute for each variable.  
We'll use metview's [`unipressure`](https://metview.readthedocs.io/en/latest/api/functions/unipressure.html) to compute the pressure on model levels.  `unipressure` just needs the lnsp fieldset to compute the pressure levels.  

In [15]:
pres_gg = mv.unipressure(surface_gg.select(shortName="lnsp"))
pres_gg.describe()

parameter,typeOfLevel,level,date,time,step,number,paramId,class,stream,type,experimentVersionNumber
pres,hybrid,"1,2,...",20171009,600,0,None,54,None,None,None,None


# Thermodynamic computations

While specific humidity is the best variable to accurately represent the amount of water present in the atmosphere, there are other ways to express that.  The most commonly used are relative humidity and dewpoint temperature.  To calculate relative humidity, metview has the [`relative_humidity_from_specific_humidity`](https://metview.readthedocs.io/en/latest/api/functions/relative_humidity_from_specific_humidity.html) function.  We can use [`dewpoint_from_specific_humidity(`](https://metview.readthedocs.io/en/latest/api/functions/dewpoint_from_specific_humidity.html) to calculate the dewpoint temperature. 

In [16]:
t_gg = wind_gg.select(shortName='t')
q_gg = moist_fieldset.select(shortName='q')
lnsp_gg = surface_gg.select(shortName="lnsp")
zs_gg = surface_gg.select(shortName="z")

r_gg = mv.relative_humidity_from_specific_humidity(t_gg,q_gg,lnsp_gg)
td_gg = mv.dewpoint_from_specific_humidity(q_gg,lnsp_gg)

moist_fieldset = mv.merge(moist_fieldset,r_gg,td_gg)

# Computing Geopotential

From the [ECMWF IFS Documentation](https://www.ecmwf.int/en/elibrary/16647-ifs-documentation-cy41r2-part-iii-dynamics-and-numerical-procedures), we know that the geopotential, $\phi$ is defined such that $\frac{\partial\phi}{\partial\eta} = -\frac{R_{d}T_{v}}{p}\frac{\partial p}{\partial\eta}$, where $\eta$ is the model level, $R_d$ is the dry gas constant, and $T_v$ is the virtual temperature, $T_{v} = \left[1 + \left(\frac{R_{vap}}{R_{d}} - 1\right)q\right]T$, where q is the specific humidity, and $R_{vap}$ is the water vapor gas constant.  Instead of vertically integrating this ourselves, we can calculate it using [mvl_geopotential_on_ml](https://metview.readthedocs.io/en/latest/api/functions/mvl_geopotential_on_ml.html) .


In [17]:
zm_gg = mv.mvl_geopotential_on_ml(t_gg, q_gg, lnsp_gg, zs_gg)
pz_fieldset = mv.merge(pres_gg, lnsp_gg, zm_gg)

del q_gg
del t_gg
del r_gg
del td_gg
del zm_gg

# Collation and conversion
Now the last steps are to collect all of the reduced-Gaussian grid datasets together and interpolate them to a regular latitude-longitude grid.

In [18]:
gg_fieldset = mv.merge(wind_gg, pz_fieldset, moist_fieldset)
#gg_fieldset = mv.merge(gg_fieldset, moist_fieldset)
del wind_gg
del pz_fieldset
del moist_fieldset

ll_fieldset = mv.read(data=gg_fieldset, grid=[0.25, 0.25])
ll_fieldset = mv.merge(ll_fieldset, uv_wind)

In [19]:
ll_fieldset.describe()

parameter,typeOfLevel,level,date,time,step,number,paramId,class,stream,type,experimentVersionNumber
cc,hybrid,"1,2,...",20171009,600,0,None,248,None,None,None,None
ciwc,hybrid,"1,2,...",20171009,600,0,None,247,None,None,None,None
clwc,hybrid,"1,2,...",20171009,600,0,None,246,None,None,None,None
crwc,hybrid,"1,2,...",20171009,600,0,None,75,None,None,None,None
cswc,hybrid,"1,2,...",20171009,600,0,None,76,None,None,None,None
d,hybrid,"1,2,...",20171009,600,0,None,155,None,None,None,None
dpt,hybrid,"1,2,...",20171009,600,0,None,3017,None,None,None,None
lnsp,hybrid,1,20171009,600,0,None,152,None,None,None,None
o3,hybrid,"1,2,...",20171009,600,0,None,203,None,None,None,None
pres,hybrid,"1,2,...",20171009,600,0,None,54,None,None,None,None


# Plotting with Metview

In Metview, the view is the fundamental concept defining the plot of a variable.  It can be a vertical profile, a vertical cross-section or a horizontal cross-section.  We will start with a horizonal plot of a single model level using geographical coordinates, this is known as a [geoview](https://metview.readthedocs.io/en/latest/gen_files/icon_functions/geoview.html). Geoviews can be specified using lat-lon corners using `mapCorners` or using predefined `mapAreaNames` ([Gallery of mapAreaNames](https://metview.readthedocs.io/en/latest/_images/map_area_gallery.png)).  The map plotting inside of a Geoview is controlled by a [mcoast](https://metview.readthedocs.io/en/latest/gen_files/icon_functions/mcoast.html) object.  This controls such things as lat/lon grid spacing/plotting, poltical boundary plotting, and land/sea fill colors.  The plot title is controlled by a [mtext](https://metview.readthedocs.io/en/latest/gen_files/icon_functions/mtext.html) object.  The actual contour plot of the variable is defined by the [mcont](https://metview.readthedocs.io/en/latest/gen_files/icon_functions/mcont.html) object.  The plotting style can either be set using the `contour_style_name`.  Vector wind plots are controlled by the [mwind](https://metview.readthedocs.io/en/latest/gen_files/icon_functions/mwind.html) object.  

The following function plots a single field with contour lines or shaded fill, depending on the `styleName` passed in.  After this function definiton, there are several examples demonstrating different contour styles and `mcoast` options.   
(If you want to plot the state/province boundaries for a list of countires, you'll need to pass in a list of [Three-letter ISO country codes](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3) to `mapAdministrativeBoundariesCountriesList`)

[Wiki on customising Metview plot titles](https://confluence.ecmwf.int/display/METV/Customising+Your+Plot+Title
)

In [20]:
from util import shaded_plot
# Plot wind speed as a shaded contour using Beaufort scale
def ws_shaded_plot(fieldset,
                   **kwargs
                   ):
    shaded_plot(fieldset,
                shortName="ws",
                styleName="sh_all_f03t70_beauf",
               **kwargs)


# Plot temperature in Celsius with a shaded contour
def temperature_shaded_plot(fieldset,
                            **kwargs
                            ):
    shaded_plot(fieldset,
                shortName="t",
                scaleBias=-273.15,
                unitString="C",
                styleName="sh_all_fM50t58i2",
                **kwargs)

# Plot dewpoint temperature in Celsius with a shaded contour
def dewpoint_shaded_plot(fieldset,
                         **kwargs,
                         ):
    shaded_plot(fieldset,
                shortName="dpt",
                scaleBias=-273.15,
                unitString="C",
                styleName="sh_all_fM32t42i2",
                **kwargs)
    
# Plot RH with a shaded contour
def relative_humidity_shaded_plot(fieldset,
                         **kwargs,
                         ):
    shaded_plot(fieldset,
                shortName="r",
                styleName="sh_grn_f10t100lst",
                **kwargs)


# Plot pressure contours every 2 hPa
def pressure_contour_plot(fieldset,
                          **kwargs
                          ):
    shaded_plot(fieldset,
                shortName='pres',
                scaleFactor=1E-2,
                unitString="hPa",
                mapCoastLandShade="on",
                mapCoastSeaShade="on",
                styleName="ct_blk_i2_t2",
                **kwargs)

# Plot geopotential height
def gh_contour_plot(fieldset,
                       **kwargs
                       ):
    shaded_plot(fieldset,
                shortName='z',
                scaleFactor=1.0/100*1.0/9.80625,
                styleName="ct_blk_i2_t2",
                **kwargs)

# Plot vertical velocity w/shaded contours
def omega_contour_plot(fieldset,
                       **kwargs
                       ):
    shaded_plot(fieldset,
                shortName='w',
                styleName="sh_viobrn_fM5t5lst",
                **kwargs)


# Plot wind arrows on top of a shaded plot of wind speed using Beaufort scale
def ws_plot_with_arrows(fieldset,
                        **kwargs
                        ):

    shaded_plot(fieldset,
                shortName="ws",
                plotWind=True,
                styleName="sh_all_f03t70_beauf",
                **kwargs
                )

In [21]:
line = [52.607718557800936, -10.528331378290753, 50.34944374693702, -4.144843744017758]
uk_corners = [60, -15, 40, 5]
ca_corners = [50, -130, 30, -110]

In [22]:
gh_contour_plot(ll_fieldset,
                     mapCorners=ca_corners,
                     mapCoastLineThickness=2,
                     level=137)

Image(value=b'', layout="Layout(visibility='hidden')")

Label(value='Generating plots....')

In [23]:
ws_shaded_plot(ll_fieldset, mapAdministrativeBoundaries="on", 
               mapAdministrativeBoundariesCountriesList=["USA","CAN"], mapAdministrativeBoundariesThickness=5, 
               mapBoundariesThickness=5, mapRivers="off", mapLabelHeight=0.6,
               level=130)

Image(value=b'', layout="Layout(visibility='hidden')")

Label(value='Generating plots....')

In [24]:
oct_line = [54, -6, 48, 3]
tubbs_line = [37.328137611093354, -124.03167766750832, 39.512677180582735, -121.46521153221067]
ws_plot_with_arrows(ll_fieldset,
                    mapCorners=[40, -124.5, 36, -121],
                    mapAdministrativeBoundaries="on",
                    mapAdministrativeBoundariesCountriesList=["USA", "CAN"], 
                    mapAdministrativeBoundariesThickness=5,
                    mapBoundariesThickness=5,
                    line=tubbs_line,
                    plotLine=True,
                    coastLatSpacing=2.0,
                    coastLonSpacing=1.5,
                    windFieldType="flags"
                    )

Image(value=b'', layout="Layout(visibility='hidden')")

Label(value='Generating plots....')

In [25]:

gh_contour_plot(ll_fieldset,
                    mapCorners=[40, -124.5, 36, -121],
                    mapAdministrativeBoundaries="on",
                    mapAdministrativeBoundariesCountriesList=["USA", "CAN"], 
                    mapAdministrativeBoundariesThickness=5,
                    mapBoundariesThickness=5,
                    line=tubbs_line,
                    plotLine=True,
                    coastLatSpacing=2.0,
                    coastLonSpacing=1.5,
                    windFieldType="flags",
                    mapRivers="on"
                    )

Image(value=b'', layout="Layout(visibility='hidden')")

Label(value='Generating plots....')

In [26]:
def thermodynamic_profile(fieldset,
                          coordinates=[0, 0],
                          diagramType="skewt", # Choices are "skewt", "tephigram", "emagram"
                          parcel_mode="most_unstable", # Choices are "surface","most_unstable"
                          top_pressure=100.0,
                          outputWidth=900
                          ):

    profile_data = mv.thermo_grib(coordinates=coordinates,
                                  data=fieldset.select(
                                      shortName=["t", "q", "u", "v", "lnsp"])
                                  )
    parcel = mv.thermo_parcel_path(profile_data, {"mode": parcel_mode})

    # create plot object for parcel areas and path
    parcel_area = mv.thermo_parcel_area(parcel)
    parcel_vis = mv.xy_curve(parcel["t"], parcel["p"], "charcoal", "dash", 6)

    # define temperature and dewpoint profile style
    profile_vis = mv.mthermo(
        thermo_temperature_line_thickness=5, thermo_dewpoint_line_thickness=5
    )

    # define a skew-T thermodynamic diagram view
    view = mv.thermoview(type=diagramType,
                         top_pressure=top_pressure)

    # define text lines for info box
    txt = []
    txt.append("     Mode: " + parcel["start"]["mode"])
    txt.append("  Start p: {:.0f} hPa".format(parcel["start"]["p"]))
    txt.append("  Start t: {:.1f} C".format(parcel["start"]["t"]))
    txt.append(" Start td: {:.1f} C".format(parcel["start"]["td"]))
    txt.append("     CAPE: {:.3f} J/kg".format(parcel["cape"]))
    txt.append("      CIN: {:.3f} J/kg".format(parcel["cin"]))

    if parcel["lcl"] is not None:
        txt.append("    LCL p: {:.0f} hPa".format(parcel["lcl"]["p"]))
        txt.append("    LCL t: {:.1f} C".format(parcel["lcl"]["t"]))

    if parcel["lfc"] is not None:
        txt.append("    LFC p: {:.0f} hPa".format(parcel["lfc"]["p"]))
        txt.append("    LFC t: {:.1f} C".format(parcel["lfc"]["t"]))

    if parcel["el"] is not None:
        txt.append("     EL p: {:.0f} hPa".format(parcel["el"]["p"]))
        txt.append("     EL t: {:.1f} C".format(parcel["el"]["t"]))

    if parcel["top"] is not None:
        txt.append("    TOP p: {:.0f} hPa".format(parcel["top"]["p"]))
        txt.append("    TOP t: {:.1f} C".format(parcel["top"]["t"]))

    # create info box - make sure font is monospace
    info_box = mv.mtext(
        text_lines=txt,
        text_font="courier",
        text_font_size=0.40,
        text_colour="charcoal",
        text_justification="left",
        text_mode="positional",
        text_box_x_position=14.8,
        text_box_y_position=11.4,
        text_box_x_length=5.4,
        text_box_y_length=len(txt) * 0.45 + 0.4,
        text_box_blanking="on",
        text_border="on",
        text_border_colour="charcoal",
    )

    info = mv.thermo_data_info(profile_data)
    title_text = "ERA5 Profile {} {:04d} UTC at Latitude: {} Longitude: {}".format(
        int(info['date']), int(info['time']), info['lat'], info['lon'])

    title = mv.mtext(
        text_font_size=0.8,
        text_lines=[
            title_text,
        ],
        text_colour="CHARCOAL",
    )

    mv.setoutput("jupyter", output_width=outputWidth)

    # plot the profile, parcel areas and parcel path together
    mv.plot(view, parcel_area, profile_data, profile_vis, parcel_vis, info_box, title)
    del (profile_data)

In [27]:
koun_coords=[35.18, -97.44]
santa_rosa=[38.44, -122.71]
thermodynamic_profile(ll_fieldset,coordinates=[38.44, -122.71], 
                      diagramType="skewt", 
                      top_pressure=100.0)

Image(value=b'', layout="Layout(visibility='hidden')")

Label(value='Generating plots....')

In [28]:
def hodograph_plot(fieldset,
                   coordinates=[0, 0],
                   speed_increment=10,
                   max_windspeed=-1,
                   top_pressure=50.0,
                   outputWidth=900):

    import math
    import numpy as np
    profile_data = mv.thermo_grib(coordinates=coordinates,
                                  data=fieldset.select(
                                      shortName=["t", "q", "u", "v", "lnsp"])
                                  )
    wind_data = mv.thermo_data_values(profile_data,0)
    info = mv.thermo_data_info(profile_data)
    p = wind_data["p_wind"]
    u = wind_data["u"]
    v = wind_data["v"]

    u_arr = np.array(u)
    v_arr = np.array(v)
    si10_arr = np.sqrt(np.power(u_arr,2) + np.power(v_arr,2))
    
    if (max_windspeed < 0):
        max_windspeed = si10_arr.max()
        
    if (max_windspeed < 10):
        max_windspeed=15
        
    print(max_windspeed)
    
    # define the hodograph background
    hodo_incr = speed_increment
    
    hodo_highlight = np.arange(10,max_windspeed,speed_increment*2)
    hodo_labels = list(np.arange(10,max_windspeed,speed_increment))
    hodo_max = max_windspeed
    hodo_colour = "black"

    # define the wind speed bins and their associated colours
    pres_bins = [1050, 700, 500, 300, 200, 50]
    pres_colours = ["red", "kelly_green", "sky", "blue", "magenta"]

    # define horizontal and vertical  axes
    h_axis = mv.maxis(axis_position="left", axis_tick_label_height=0.8)
    v_axis = mv.maxis(axis_position="bottom", axis_tick_label_height=0.4)

    # the view
    view = mv.cartesianview(
        x_automatic="off",
        x_min=-hodo_max,
        x_max=hodo_max,
        y_automatic="off",
        y_min=-hodo_max,
        y_max=hodo_max,
        horizontal_axis=h_axis,
        vertical_axis=h_axis,
        subpage_x_position=10,
        subpage_y_position=5,
        subpage_x_length=80,
        subpage_y_length=80,
    )
    
    
    # size is in % of the physical size of the superpage!
    hodo_page = mv.plot_page(top=5, bottom=95, left=5, right=95, view=view)

    # size is in cm!
    dw = mv.plot_superpage(
        layout_size="custom", custom_width=15, custom_height=15, pages=hodo_page
    )

    gr_lst = []

    # build the concentric circles
    sp = hodo_incr
    angle_incr = 2 * math.pi / 180
    while sp <= hodo_max:
        xp = [math.cos(i * angle_incr) * sp for i in range(1, 182)]
        yp = [math.sin(i * angle_incr) * sp for i in range(1, 182)]

        if sp in hodo_highlight:
            gr = mv.xy_curve(xp, yp, hodo_colour, "solid", 3)
        else:
            gr = mv.xy_curve(xp, yp, hodo_colour, "solid", 1)

        gr_lst.append(gr)
        sp += hodo_incr

    # build horizontal and vertical lines going
    # throug the centre
    gr_lst.append(mv.xy_curve([-hodo_max, hodo_max],
                  [0, 0], hodo_colour, "solid", 1))
    gr_lst.append(mv.xy_curve(
        [0, 0], [-hodo_max, hodo_max], hodo_colour, "solid", 1))

    # build labels on the horizontal line
    vis = mv.input_visualiser(
        input_plot_type="xy_point",
        input_x_values=[-v for v in hodo_labels] + hodo_labels,
        input_y_values=[0 for i in range(len(hodo_labels) * 2)],
        input_values=hodo_labels + hodo_labels,
    )

    sym = mv.msymb(
        symbol_colour=hodo_colour,
        symbol_text_font_size=0.8,
        symbol_text_font_style="bold",
        symbol_text_position="bottom",
    )

    gr_lst.extend([vis, sym])

    # build the graphical objects for the wind data (per bin)
    gr_wind = []
    for i in range(len(pres_bins) - 1):

        # collect wind data in bin
        u_val = []
        v_val = []
        p_val = []
        u_next = -9999
        v_next = -9999
        cont_flag = True
        for k in range(len(p)):
            if (
                not math.isnan(p[k])
                and not math.isnan(u[k])
                and not math.isnan(v[k])
                and p[k] <= pres_bins[i]
                and p[k] >= pres_bins[i + 1]
                and p[k] >= top_pressure
            ):
                u_val.append(u[k])
                v_val.append(v[k])
                p_val.append(p[k])
                if (k<136):
                    u_next = u[k+1]
                    v_next = v[k+1]
                    p_next = p[k+1]
            else:
                if (u_next > -9999):
                    u_val.append(u_next)
                    v_val.append(v_next)
                    p_val.append(p_next)
            
        # build graph object
        if u_val and v_val:
            vis = mv.input_visualiser(
                input_x_values=u_val, input_y_values=v_val)

            gr = mv.mgraph(
                legend="on",
                graph_line_colour=pres_colours[i],
                graph_line_style="solid",
                graph_line_thickness=5,
            )
            gr_wind.extend([vis, gr])

    # define legend with custom labels
    legend_text = []
    for i in range(len(pres_bins) - 1):
        legend_text.append(str(pres_bins[i]) + "-" + str(pres_bins[i + 1]))

    legend = mv.mlegend(
        legend_display_type="disjoint",
        legend_text_font_size=0.8,
        legend_text_composition="user_text_only",
        legend_user_lines=legend_text,
    )

    # define title
    title_txt = "ERA5 Hodograph (m s**-1) Date: {} {:04d}  Latitude: {} Longitude: {}".format(
        int(info["date"]), int(info["time"]), info["lat"], info["lon"]
    )

    title = mv.mtext(text_lines=title_txt, text_font_size=0.8,
                     text_colour="charcoal")

    # generate the plot
    mv.plot(dw, gr_lst, gr_wind, legend, title)

In [29]:
hodograph_plot(ll_fieldset,coordinates=santa_rosa, speed_increment=10)

69.32073710948848


Image(value=b'', layout="Layout(visibility='hidden')")

Label(value='Generating plots....')

## Pressure cross-sections

In [30]:
def pressure_temp_ws_cross_section(fieldset,
                                   line=[0, 0, 0, 10],
                                   top_level=50.0,
                                   outputWidth=1800,
                                   ):
    t = ll_fieldset.select(shortName='t') - 273.15
    u = ll_fieldset.select(shortName='u')
    v = ll_fieldset.select(shortName='v')
    sp = mv.speed(u, v)
    print(mv.maxvalue(sp))
    del (u)
    del (v)
    lnsp = ll_fieldset.select(shortName='lnsp')

    # define shading for wind speed
    sp_cont = mv.mcont(legend="on",
                       contour_automatics_settings="style_name",
                       grib_scaling_of_derived_fields='on',
                       contour_style_name="sh_grn_f10t100lst",
                       )

    # define contouring for temperature
    t_cont = mv.mcont(
        contour_line_style="dash",
        contour_line_thickness=2,
        contour_line_colour="charcoal",
        contour_highlight="off",
        contour_level_selection_type="interval",
        contour_label_height=0.5,
        contour_interval=5,
    )

    vertical_axis = mv.maxis(
        axis_orientation="vertical",
        axis_type="position_list",
        axis_tick_position_list=[1000, 850, 700, 500, 400, 300, 200, 100, 50],
        axis_tick_label_height=0.7,
        axis_title_height=0.7,
        axis_title_text="Pressure (hPa)"
    )

    geo_axis = mv.maxis(
        axis_orientation="horizontal",
        axis_type="geoline",
        axis_tick_label_height=0.6,
        axis_title_text="Position (Lat/Lon)",
        axis_title_height=0.6,
    )

    # define cross section in log pressure from surface to top_level
    xs_view = mv.mxsectview(line=line, top_level=top_level,
                            horizontal_axis=geo_axis,
                            vertical_axis=vertical_axis)

    # define orography area
    orog_graph = mv.mgraph(
        graph_type="area",
        graph_shade_colour="charcoal",
    )

    # define cross section data (field + lnsp)
    xs_t_data = mv.merge(t, lnsp)
    xs_sp_data = mv.merge(sp, lnsp)

    # define the legend (so we can control the font size)
    legend = mv.mlegend(
        legend_text_font_size=0.5
    )

    # get metadata for the title
    meta = mv.grib_get(t[0], ["date", "time"])[0]

    # set-up the title
    title_text = "ERA5 Temperature (C) (contour) and Wind Speed (m/s) (shaded) {} {} UTC".format(
            meta[0], meta[1])
    title = mv.mtext(
        text_font_size=0.7,
        text_lines=[
            title_text,
        ],
        text_colour="CHARCOAL",
    )

    # generate plot
    mv.setoutput("jupyter", output_width=outputWidth)
    mv.plot(xs_view, xs_sp_data, sp_cont, xs_t_data,
            t_cont, orog_graph, legend, title)

    for item in [t, sp, lnsp]:
        del (item)

In [31]:
def pressure_temp_rh_cross_section(fieldset,
                           line=[0, 0, 0, 10],
                           top_level=50.0,
                           outputWidth=1800,
                           ):
    t = ll_fieldset.select(shortName='t') - 273.15
    rh = ll_fieldset.select(shortName='r')
    lnsp = ll_fieldset.select(shortName='lnsp')

    # define shading for RH
    rh_cont = mv.mcont(legend="on",
                       contour_automatics_settings="style_name",
                       grib_scaling_of_derived_fields='on',
                       #contour_style_name="sh_grn_f10t100lst",
                       contour_style_name="sh_grnblu_f65t100i15"
                       )

    # define contouring for temperature
    t_cont = mv.mcont(
        contour_line_style="dash",
        contour_line_thickness=2,
        contour_line_colour="charcoal",
        contour_highlight="off",
        contour_level_selection_type="interval",
        contour_label_height = 0.5,
        contour_interval=5,
    )

    vertical_axis = mv.maxis(
        axis_orientation="vertical",
        axis_type="position_list",
        axis_tick_position_list=[1000, 850, 700, 500, 400, 300, 200, 100, 50],
        axis_tick_label_height=0.7,
        axis_title_height=0.7,
        axis_title_text="Pressure (hPa)"
    )

    geo_axis = mv.maxis(
        axis_orientation="horizontal",
        axis_type="geoline",
        axis_tick_label_height=0.6,
        axis_title_text="Position (Lat/Lon)",
        axis_title_height=0.6,
    )

    # define cross section in log pressure from surface to top_level
    xs_view = mv.mxsectview(line=line, top_level=top_level,
                            horizontal_axis=geo_axis,
                            vertical_axis=vertical_axis)

    # define orography area
    orog_graph = mv.mgraph(
        graph_type="area",
        graph_shade_colour="charcoal",
    )

    # define cross section data (field + lnsp)
    xs_t_data = mv.merge(t, lnsp)
    xs_rh_data = mv.merge(rh, lnsp)

    # define the legend (so we can control the font size)
    legend = mv.mlegend(
        legend_text_font_size=0.5
    )

    # get metadata for the title
    meta = mv.grib_get(t[0], ["date", "time"])[0]

    # set-up the title
    title_text = "ERA5 Temperature (C) (contour) and Relative Humidity (%) (shaded) {} {} UTC".format(
            meta[0], meta[1])

    title = mv.mtext(
        text_font_size=0.7,
        text_lines=[
            title_text,
        ],
        text_colour="CHARCOAL",
    )
    
    # generate plot
    mv.setoutput("jupyter", output_width=outputWidth)
    mv.plot(xs_view, xs_rh_data, rh_cont, xs_t_data, t_cont, orog_graph, legend, title)

    for item in [t, rh, lnsp]:
        del (item)

In [32]:
pressure_temp_ws_cross_section(ll_fieldset,line=tubbs_line)

127.6770362406969


Image(value=b'', layout="Layout(visibility='hidden')")

Label(value='Generating plots....')

## Model-level cross-sections

In [33]:
def model_level_winds_cross_section(fieldset,
                                    line=[0, 0, 0, 10],
                                    topLevel=3000,
                                    levelSpacing=100,
                                    w_scale_factor=100,
                                    outputWidth=1800):

    # extract model level data
    u = fieldset.select(shortName='u')
    v = fieldset.select(shortName='v')
    t = fieldset.select(shortName='t')
    omega = fieldset.select(shortName='w')
    # convert to geopotential height
    z = fieldset.select(shortName='z')/9.80625
    lnsp = fieldset.select(shortName='lnsp')
    zs = z.select(level=137)

    # vertical velocity in m/s
    w = mv.w_from_omega(omega, t, lnsp)
    w_max = w.maxvalue()
    w_min = w.minvalue()
    print("W max:{} min:{}".format(w_max, w_min))

    f_uv = mv.merge(u, v, w, z)
    t = t - 273.15

    # define bottom and top level (m)
    top_level = topLevel
    bottom_level = 0

    # define number of target levels
    level_count = 1 + (top_level-bottom_level)/levelSpacing

    # compute cross sections in height (above sea level) for a fixed set
    # of target levels (at least Metview version 5.16.0 is required).
    
    # temperature
    xs_temp = mv.mcross_sect(
        data=mv.merge(t, z),
        line=line,
        level_selection_type="count",
        level_count=level_count,
        vertical_coordinates="user",
        vertical_coordinate_param=129,
        vertical_coordinate_extrapolate="on",
        vertical_coordinate_extrapolate_mode="constant",
        top_level=top_level,
        bottom_level=bottom_level,
    )

    # Winds
    xs_wind = mv.mcross_sect(
        data=f_uv,
        line=line,
        wind_parallel="on",
        wind_perpendicular="off",
        w_wind_scaling_factor_mode="user",
        w_wind_scaling_factor=w_scale_factor,
        level_selection_type="count",
        level_count=level_count,
        vertical_coordinates="user",
        vertical_coordinate_param=129,
        vertical_coordinate_extrapolate="off",
        vertical_coordinate_extrapolate_mode="constant",
        top_level=top_level,
        bottom_level=bottom_level,
    )
    # generate orography area curve
    orog_curve = mv.xs_build_orog(xs_wind, zs, bottom_level, "charcoal")

    # define wind plotting style
    wind_style = mv.mwind(
        wind_thinning_factor=1, wind_arrow_colour="navy", wind_arrow_unit_velocity=30
    )

    # define contour shading for  wind component
    wind_shade = mv.mcont(
        legend="on",
        contour="off",
        contour_level_selection_type="interval",
        contour_shade_max_level=30,
        contour_shade_min_level=-30,
        contour_interval=4,
        contour_label="off",
        contour_shade="on",
        contour_shade_colour_method="palette",
        contour_shade_method="area_fill",
        contour_shade_palette_name="colorbrewer_PuOr_18_r",
    )

    # define contouring for temperature
    solid_cont = mv.mcont(
        contour_line_style="solid",
        contour_line_colour="black",
        contour_highlight_colour="black",
        contour_highlight_thickness=2,
        contour_level_selection_type="interval",
        contour_interval=2,
        contour_label_height=0.6,
    )
    
    # define horizontal aix
    geo_axis = mv.maxis(
        axis_orientation="horizontal",
        axis_type="geoline",
        axis_tick_label_height=0.6,
        axis_title_text="(Lat/Lon)",
        axis_title_height=0.6,
    )

    # define vertical axis
    vertical_axis = mv.maxis(
        axis_orientation="vertical",
        axis_title_text="Height ASL (m)",
        axis_tick_label_height=0.8,
        axis_title_height=0.8,
        axis_title_position=140,
    )

    # define cross section view in height above sea level (m)
    xs_view = mv.mxsectview(
        line=line,
        top_level=top_level,
        bottom_level=bottom_level,
        horizontal_axis=geo_axis,
        vertical_axis=vertical_axis,
    )

    # define legend
    legend = mv.mlegend(legend_text_font_size=0.8)

    # get metadata for the title
    meta = mv.grib_get(w[0], ["date", "time", "step"])[0]

    # define title
    title = mv.mtext(
        text_lines=[
            f"Along line Winds",
            f"ERA5: {meta[0]} {meta[1]} UTC",
        ],
        text_font_size=0.8,
    )

    mv.setoutput("jupyter", output_width=outputWidth)
    mv.plot(
        xs_view,
        xs_wind,
        wind_style,
        xs_temp,
        solid_cont,
        orog_curve,
        legend,
        title,
    )

    for item in [t, f_uv, z, w]:
        del (item)

## MetView References
- [Gallery of mapAreaNames](https://metview.readthedocs.io/en/latest/_images/map_area_gallery.png)
- [mtext](https://metview.readthedocs.io/en/latest/gen_files/icon_functions/mtext.html)
- [mcoast](https://metview.readthedocs.io/en/latest/gen_files/icon_functions/mcoast.html)
- [thermo_grib](https://metview.readthedocs.io/en/latest/gen_files/icon_functions/thermo_grib.html)
- [thermoview](https://metview.readthedocs.io/en/latest/gen_files/icon_functions/thermoview.html)
- [mwind](https://metview.readthedocs.io/en/latest/gen_files/icon_functions/mwind.html)
- [Three-letter ISO country codes](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3)